# **Packing Spot Tracking**

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, Activation

# *Dataset*

In [ ]:
data_folder_A = './data/data2/A/'
data_folder_B = 'data/data2/B/'

In [ ]:
os.listdir(data_folder_B)

In [ ]:
h, w = 150, 150
batch_size = 100
lr = 0.0001
epoches = 20

In [ ]:
train_dataset = tf.keras.utils.image_dataset_from_directory(data_folder_A,
                                                            validation_split=0.2,
                                                            seed=123,
                                                            subset='training',
                                                            image_size=(h, w),
                                                            batch_size=batch_size
                                                           )

In [ ]:
val_dataset = tf.keras.utils.image_dataset_from_directory(data_folder_B,
                                                            validation_split=0.2,
                                                            seed=123,
                                                            subset='validation',
                                                            image_size=(h, w),
                                                            batch_size=batch_size
                                                           )

In [ ]:
plt.figure(figsize=(100, 100))
for images, labels in train_dataset.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i+1)
        plt.imshow(images[i].numpy().astype('uint8'))
        plt.axis('off')

In [ ]:
for images, labels in train_dataset:
    print(images.shape)
    print(labels.shape)

# *Preprocessing*

In [ ]:
normalized_layer = tf.keras.layers.Rescaling(1./255)

In [8]:
normalized_dataset = train_dataset.map(lambda x, y: (normalized_layer(x), y))

image_batch, labels_batch = next(iter(normalized_dataset))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.121568635 1.0


# *Config Model*

In [9]:
model = tf.keras.Sequential()

In [ ]:
model.add(Conv2D((11,11),4))

In [ ]:
model.summary()

---

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(2))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset, epochs=20, 
                    validation_data=val_dataset)

---

In [ ]:
model.compile(
  optimizer='adam',
  loss="sparse_categorical_crossentropy",
  metrics=['accuracy'])

# *Training model*

In [ ]:
history = model.fit(train_dataset, validation_data=val_dataset, epochs=50)

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# *Evaluate*